Import Library

In [3]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [ ]:
base_dir = os.path.abspath(os.getcwd()) + os.pardir("Data/Loon-colon-image")

base_dir